In [122]:
import os
os.environ['DGLBACKEND'] = 'pytorch'

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import dgl.function as fn
import dgl
from dgllife.data import Tox21
from dgllife.utils import SMILESToBigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer, RandomSplitter
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from dgl.data.utils import split_dataset
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score as rac
import torch.optim as optim
from tqdm.notebook import tqdm, trange
from scipy import signal
import cupy as cp

In [141]:
# Set device to GPU if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

smiles_to_g = SMILESToBigraph(node_featurizer=CanonicalAtomFeaturizer(), edge_featurizer=CanonicalBondFeaturizer())

dataset = Tox21(smiles_to_g)

dataset[0]

Processing dgl graphs from scratch...
Processing molecule 1000/7831
Processing molecule 2000/7831
Processing molecule 3000/7831
Processing molecule 4000/7831
Processing molecule 5000/7831
Processing molecule 6000/7831
Processing molecule 7000/7831


('CCOc1ccc2nc(S(N)(=O)=O)sc2c1',
 Graph(num_nodes=16, num_edges=34,
       ndata_schemes={'h': Scheme(shape=(74,), dtype=torch.float32)}
       edata_schemes={'e': Scheme(shape=(12,), dtype=torch.float32)}),
 tensor([0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 tensor([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1.]))

In [148]:
# Batching a list of datapoints for dataloader.
def collate_molgraphs(data):
    smiles, graphs, labels, masks = map(list, zip(*data))
    
    # Add self-loops to each individual graph
    for i in range(len(graphs)):
        graphs[i] = dgl.add_self_loop(graphs[i])
    
    batched_graphs = dgl.batch(graphs)
    batched_graphs.set_n_initializer(dgl.init.zero_initializer)
    batched_graphs.set_e_initializer(dgl.init.zero_initializer)

    labels = torch.stack(labels, dim=0).to(device)
    masks = torch.stack(masks, dim=0).to(device)
    return smiles, batched_graphs.to(device), labels, masks


train_set, val_set, test_set = split_dataset(dataset, shuffle=True)
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, collate_fn=collate_molgraphs)
val_loader = DataLoader(val_set, batch_size=128, shuffle=True, collate_fn=collate_molgraphs)
test_loader = DataLoader(test_set, batch_size=128, shuffle=True, collate_fn=collate_molgraphs)


In [125]:
class Meter(object):
    """Track and summarize model performance on a dataset for
    (multi-label) binary classification."""

    def __init__(self):
        self.mask = []
        self.y_pred = []
        self.y_true = []

    def update(self, y_pred, y_true, mask):
        """Update for the result of an iteration
        Parameters
        ----------
        y_pred : float32 tensor
            Predicted molecule labels with shape (B, T),
            B for batch size and T for the number of tasks
        y_true : float32 tensor
            Ground truth molecule labels with shape (B, T)
        mask : float32 tensor
            Mask for indicating the existence of ground
            truth labels with shape (B, T)
        """
        self.y_pred.append(y_pred.detach().cpu())
        self.y_true.append(y_true.detach().cpu())
        self.mask.append(mask.detach().cpu())

    def roc_auc_score(self):
        """Compute roc-auc score for each task.
        Returns
        -------
        list of float
            roc-auc score for all tasks
        """
        mask = torch.cat(self.mask, dim=0)
        y_pred = torch.cat(self.y_pred, dim=0)
        y_true = torch.cat(self.y_true, dim=0)
        # This assumes binary case only
        y_pred = torch.sigmoid(y_pred)
        return [
            rac(y_pred[:, i], y_true[:, i], sample_weight=mask[:, i])
            for i in range(y_true.size(1))
        ]

In [126]:
def run_an_eval_epoch(model, loader):
    model.eval()
    meter = Meter()

    with torch.no_grad():
        for smiles, bg, labels, masks in loader:
            labels, masks = labels.to(device), masks.to(device)
            labels, masks = labels.float(), masks.float()
            logits = model(bg)
            logits = logits.unsqueeze(-1)
            loss = criterion(logits, labels)
            meter.update(logits, labels, masks)

    return meter.roc_auc_score()

In [127]:
def run_a_train_epoch(model, optimizer, loader):
    model.train()
    meter = Meter()

    for smiles, bg, labels, masks in loader:
        labels, masks = labels.to(device), masks.to(device)
        labels, masks = labels.float(), masks.float()
        logits = model(bg)
        logits = logits.unsqueeze(-1)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        meter.update(logits, labels, masks)

    return meter.roc_auc_score()

In [128]:
class GATLayer1(nn.Module):
    def __init__(self, num_in_feats, num_out_feats, num_heads):
        super(GATLayer1, self).__init__()
        self.gat = dgl.nn.pytorch.GATConv(num_in_feats, num_out_feats, num_heads,
                                          residual=False)

    def forward(self, g, h):
        with g.local_scope():
            h = self.gat(g, h)
            return h

In [129]:
class MultiHeadGATLayer(nn.Module):
    def __init__(self, num_in_feats, num_out_feats, num_heads):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for _ in range(num_heads):
            self.heads.append(GATLayer1(num_in_feats, num_out_feats, 1))

    def forward(self, g, h):
        head_outs = [attn_head(g, h) for attn_head in self.heads]
        # return the concatenated multi-head representation
        return torch.cat(head_outs, dim=1)

In [130]:
class GAT1(nn.Module):
    def __init__(self, num_layers, num_feats, num_hidden, num_classes, num_heads):
        super(GAT1, self).__init__()
        self.gat_layers = nn.ModuleList()
        self.gat_layers.append(MultiHeadGATLayer(num_feats, num_hidden, num_heads))
        for _ in range(num_layers - 1):
            self.gat_layers.append(MultiHeadGATLayer(num_hidden * num_heads, num_hidden, num_heads))
        self.gat_layers.append(MultiHeadGATLayer(num_hidden * num_heads, num_classes, num_heads))
        self.linear = nn.Linear(num_hidden * num_heads, num_classes)

    def forward(self, g):
        h = g.ndata['h'].float().to(device)
        for gat_layer in self.gat_layers:
            h = gat_layer(g, h).to(device)
        return h

num_layers = 3
num_feats = 74
num_hidden = 256
num_classes = 12
num_heads = 2

model = GAT1(num_layers, num_feats, num_hidden, num_classes, num_heads).to(device)

In [131]:
print(model)

GAT1(
  (gat_layers): ModuleList(
    (0): MultiHeadGATLayer(
      (heads): ModuleList(
        (0-1): 2 x GATLayer1(
          (gat): GATConv(
            (fc): Linear(in_features=74, out_features=256, bias=False)
            (feat_drop): Dropout(p=0.0, inplace=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (leaky_relu): LeakyReLU(negative_slope=0.2)
          )
        )
      )
    )
    (1-2): 2 x MultiHeadGATLayer(
      (heads): ModuleList(
        (0-1): 2 x GATLayer1(
          (gat): GATConv(
            (fc): Linear(in_features=512, out_features=256, bias=False)
            (feat_drop): Dropout(p=0.0, inplace=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (leaky_relu): LeakyReLU(negative_slope=0.2)
          )
        )
      )
    )
    (3): MultiHeadGATLayer(
      (heads): ModuleList(
        (0-1): 2 x GATLayer1(
          (gat): GATConv(
            (fc): Linear(in_features=512, out_features=12, bias=False)
         

In [132]:
criterion = BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [133]:
epochs = 50
best_val_score = 0
best_model = None

In [150]:
# Add self-loops to the graph
bg = dgl.add_self_loop(bg)

for epoch in trange(epochs):
    train_score = run_a_train_epoch(model, optimizer, train_loader)
    val_score = run_an_eval_epoch(model, val_loader)
    scheduler.step(val_score[0])
    if val_score[0] > best_val_score:
        best_val_score = val_score[0]
        best_model = model.state_dict()

NameError: name 'bg' is not defined

In [ ]:
model.load_state_dict(best_model)
test_score = run_an_eval_epoch(model, test_loader)
print('Test ROC-AUC Score: %.4f' % test_score[0])